In [1]:
from parser import TrainingConfig, ModelConfig
from models.unet_small import UNetSmall
from DataLoader import CustomDataLoader
training_config = TrainingConfig()
model_config = ModelConfig()

In [2]:
import numpy as np


def residual_mass(ux_matrix:np.ndarray,uy_matrix:np.ndarray):
    '''
    Compute the residual: mass conservation
    Formula: 
    Rs_mass = {d(ux)/dx + d(uy)/dy}^2.sum()/40000

    Arguments:
    ux_matrix: np.ndarray: matrix of x-velocity, shape = [200,200]
    uy_matrix: np.ndarray: matrix of y-velocity, shape = [200,200]

    Return:
    Rs_mass_sum: float: sum of Rs_mass
    '''
    ux_with_down_boundary = ux_matrix[2:200,1:199]
    ux_with_up_boundary = ux_matrix[0:198,1:199]
    uy_with_right_boundary = uy_matrix[1:199,2:200]
    uy_with_left_boundary = uy_matrix[1:199,0:198]

    pinn_dudx = (ux_with_down_boundary - ux_with_up_boundary)/(2*0.005)
    pinn_dvdy = (uy_with_right_boundary - uy_with_left_boundary)/(2*0.005)

    Rs_mass = pinn_dudx+pinn_dvdy
    Rs_mass_sq = Rs_mass*Rs_mass
    Rs_mass_sum = Rs_mass_sq.sum()/40000
    return Rs_mass_sum

In [3]:
data_path = "/home/ubuntu/ml-convection/dataset"
dataloader = CustomDataLoader(data_path, training_config)
dataloader.set_required_data()
train_loader = dataloader.get_data("train")
val_loader =  dataloader.get_data("val")

In [4]:
dataloader.max_value, dataloader.min_value

([306.89, 0.304527, 0.377805], [288.495, -0.188993, -0.234059])

In [5]:
import pytorch_lightning as pl 
import torch
import torch.nn.functional as F

import numpy as np


def residual_mass(ux_matrix:np.ndarray,uy_matrix:np.ndarray):
    '''
    Compute the residual: mass conservation
    Formula: 
    Rs_mass = {d(ux)/dx + d(uy)/dy}^2.sum()/40000

    Arguments:
    ux_matrix: np.ndarray: matrix of x-velocity, shape = [200,200]
    uy_matrix: np.ndarray: matrix of y-velocity, shape = [200,200]

    Return:
    Rs_mass_sum: float: sum of Rs_mass
    '''
    ux_matrix = ux_matrix * (dataloader.max_value[1] - dataloader.min_value[1]) + dataloader.min_value[1]
    uy_matrix = uy_matrix * (dataloader.max_value[2] - dataloader.min_value[2]) + dataloader.min_value[2]
    ux_with_down_boundary = ux_matrix[:, 2:200,1:199]
    ux_with_up_boundary = ux_matrix[:, 0:198,1:199]
    uy_with_right_boundary = uy_matrix[:,1:199,2:200]
    uy_with_left_boundary = uy_matrix[:,1:199,0:198]

    pinn_dudx = (ux_with_down_boundary - ux_with_up_boundary)/(2*0.005)
    pinn_dvdy = (uy_with_right_boundary - uy_with_left_boundary)/(2*0.005)

    Rs_mass = pinn_dudx+pinn_dvdy
    Rs_mass_sq = Rs_mass*Rs_mass
    Rs_mass_sum = Rs_mass_sq.sum()/(40000*ux_matrix.shape[0])
    return Rs_mass_sum

def residual_momentum(ux_matrix, ux_matrix_prev, uy_matrix, t_matrix):
    '''
    Compute the residual: momentum conservation
    Formula:
    Rs_mom = {d(ux)/dt + ux*d(ux)/dx + uy*d(ux)/dy - 1.831e-05/(348.33/alpha)*d^2(ux)/dx^2 - 9.81/293*(293-alpha)}^2.sum()/40000
    '''
    t_matrix = t_matrix * (dataloader.max_value[0] - dataloader.min_value[0]) + dataloader.min_value[0]
    ux_matrix = ux_matrix * (dataloader.max_value[1] - dataloader.min_value[1]) + dataloader.min_value[1]
    ux_matrix_prev = ux_matrix_prev * (dataloader.max_value[1] - dataloader.min_value[1]) + dataloader.min_value[1]
    uy_matrix = uy_matrix * (dataloader.max_value[2] - dataloader.min_value[2]) + dataloader.min_value[2]
    mom_1 = ux_matrix[:,1:199,1:199] - ux_matrix_prev[:,1:199,1:199]
    mom_3 = ux_matrix[:,1:199,1:199]*(ux_matrix[:,2:200,1:199] - ux_matrix[:,0:198,1:199])
    mom_4 = uy_matrix[:,1:199,1:199]*(ux_matrix[:,1:199,2:200] - ux_matrix[:,1:199,0:198])
    mom_5_2 = ux_matrix[:,1:199,2:200] - 2*ux_matrix[:,1:199,1:199] + ux_matrix[:,1:199,0:198] 
    mom_5 = 1.831e-05/(348.33/t_matrix[:,1:199,1:199])*(mom_5_2)
    mom_6 = 9.81/293*(293-t_matrix[:,1:199,1:199])

    Rs_mom = mom_1/0.01 +  mom_3/(2*0.005) + mom_4/(2*0.005) - mom_5/(0.005*0.005) - mom_6
    Rs_mom_sq = Rs_mom*Rs_mom
    Rs_mom_sum = Rs_mom_sq.sum()/(40000*ux_matrix.shape[0])
    return Rs_mom_sum

def residual_heat(ux_matrix:np.ndarray, uy_matrix:np.ndarray, t_matrix:np.ndarray, t_matrix_prev:np.ndarray):
    '''
    Compute the residual: heat conservation
    Formula:
    Rs_heat = {d(t)/dt + ux*d(t)/dx + uy*d(t)/dy - 0.14*(t-293)+21.7/1e6*d^2(t)/dx^2}^2.sum()/40000
    TODO: Check the formula

    Arguments:
    ux_matrix: np.ndarray: matrix of x-velocity, shape = [200,200]
    uy_matrix: np.ndarray: matrix of y-velocity, shape = [200,200]
    t_matrix: np.ndarray: matrix of temperature, shape = [200,200]
    t_matrix_prev: np.ndarray: matrix of temperature at previous time step, shape = [200,200]

    Return:
    Rs_heat_sum: float: sum of Rs_heat
    '''
    tdiff_matrix = (0.14*(t_matrix[:,1:199,1:199] - 293)+ 21.7)/1000000
    heat_1 = t_matrix[:,1:199,1:199] - t_matrix_prev[:,1:199,1:199]
    heat_2 = (t_matrix[:,2:200,1:199] - t_matrix[:,0:198,1:199])*(ux_matrix[:,1:199,1:199])
    heat_3 = (t_matrix[:,1:199,2:200] - t_matrix[:,1:199,0:198])*(uy_matrix[:,1:199,1:199])
    heat_4 = tdiff_matrix*(t_matrix[:,1:199,2:200] - 2*t_matrix[:,1:199,1:199] + t_matrix[:,1:199,0:198])

    Rs_heat = heat_1/0.01 + heat_2/(2*0.005) + heat_3/(2*0.005) - heat_4/(0.005*0.005)
    Rs_heat_sq = Rs_heat*Rs_heat
    Rs_heat_sum = Rs_heat_sq.sum()/(40000* ux_matrix.shape[0])
    return Rs_heat_sum

def unnormalize_y(unnorm_arr, min_value, max_value):
    unnorm_arr[:, 0, :, :] = (
        unnorm_arr[:, 0, :, :] * (max_value[0] - min_value[0]) + min_value[0]
    )
    unnorm_arr[:, 1, :, :] = (
        unnorm_arr[:, 1, :, :] * (max_value[1] - min_value[1]) + min_value[1]
    )
    unnorm_arr[:, 2, :, :] = (
        unnorm_arr[:, 2, :, :] * (max_value[2] - min_value[2]) + min_value[2]
    )
    return unnorm_arr
class LitUnet(pl.LightningModule):
    def __init__(self, max_value, min_value):
        super().__init__()
        self.model = UNetSmall(in_channels = 3, out_channels = 3, init_features=32)
        self.max_value = max_value
        self.min_value = min_value
        self.mass_mag = 0.2
        self.momentum_mag = 0.2
        self.heat_mag = 0.2
        self.mse_mag = 1 - self.mass_mag - self.momentum_mag - self.heat_mag
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        mse_loss = F.mse_loss(y_hat, y)
        Rs_mass_sum = residual_mass(y_hat[:,0,:,:], y_hat[:,1,:,:])
        Rs_momentum_sum = residual_momentum(y_hat[:,1,:,:], x[:,1,:,:], y_hat[:,2,:,:], y_hat[:,0,:,:])
        Rs_heat_sum = residual_heat(y_hat[:,1,:,:], y_hat[:,2,:,:], y_hat[:,0,:,:], x[:,0,:,:])
        self.log("mse loss", mse_loss, on_epoch = True)
        self.log("Rs_mass_sum", Rs_mass_sum, on_epoch = True)
        self.log("Rs Momentum sum", Rs_momentum_sum, on_epoch = True)
        self.log("Rs Heat sum", Rs_heat_sum, on_epoch = True)
        total_loss = self.mse_mag*mse_loss + self.mass_mag*Rs_mass_sum + self.momentum_mag*Rs_momentum_sum + self.heat_mag*Rs_heat_sum
        self.log("total loss", total_loss, prog_bar = True, on_epoch = True)
        return total_loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        mse_loss = F.mse_loss(y_hat, y)
        Rs_mass_sum = residual_mass(y_hat[:,0,:,:], y_hat[:,1,:,:])
        Rs_momentum_sum = residual_momentum(y_hat[:,1,:,:], x[:,1,:,:], y_hat[:,2,:,:], y_hat[:,0,:,:])
        Rs_heat_sum = residual_heat(y_hat[:,1,:,:], y_hat[:,2,:,:], y_hat[:,0,:,:], x[:,0,:,:])
        self.log("Val mse loss", mse_loss, on_epoch = True)
        self.log("Val Rs_mass_sum", Rs_mass_sum, on_epoch = True)
        self.log("Val Rs Momentum sum", Rs_momentum_sum, on_epoch = True)
        self.log("Val Rs Heat sum", Rs_heat_sum, on_epoch = True)
        unnorm_y = unnormalize_y(y, self.min_value, self.max_value)
        unnorm_y_hat = unnormalize_y(y_hat, self.min_value, self.max_value)
        max_ae_t = torch.max(torch.abs(unnorm_y[:,0,:,:] - unnorm_y_hat[:,0,:,:]))
        max_ae_ux = torch.max(torch.abs(unnorm_y[:,1,:,:] - unnorm_y_hat[:,1,:,:]))
        max_ae_uy = torch.max(torch.abs(unnorm_y[:,2,:,:] - unnorm_y_hat[:,2,:,:]))
        self.log("Max AE T", max_ae_t, on_epoch = True)
        self.log("Max AE Ux", max_ae_ux, on_epoch = True)
        self.log("Max AE Uy", max_ae_uy, on_epoch = True)
    
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr = 1e-3)
        return optimizer
       

In [6]:
earlystopping_callback = pl.callbacks.EarlyStopping(monitor='total loss', patience=1000, mode='min')
trainer = pl.Trainer(
    accelerator = "gpu",
    devices = tuple([0]),
    callbacks = [earlystopping_callback],
    max_epochs=5000)
model = LitUnet(dataloader.max_value, dataloader.min_value)
trainer.fit(model, train_loader, val_loader, )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | UNetSmall | 1.9 M 
------------------------------------
1.9 M     Trainable params
0         Non-trainable params
1.9 M     Total params
7.708     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/ubuntu/miniforge3/envs/fluid_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/ubuntu/miniforge3/envs/fluid_env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5000` reached.
